In [9]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['속성 기반 분류', '도메인 식별']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            for raw_data in line:
                #### data preprocess
                text = raw_data['RawText']
                domain = raw_data['Source']+'/'+raw_data['Domain']+'/'+raw_data['MainCategory']
                if 'GeneralPolarity' in raw_data:
                    generl_polarity = raw_data['GeneralPolarity']
                else:
                    generl_polarity = [aspect['SentimentPolarity'] for aspect in raw_data['Aspects']]
                    generl_polarity = [float(polarity) for polarity in generl_polarity]
                    generl_polarity = sum(generl_polarity) / len(generl_polarity)
                    generl_polarity = str(int(generl_polarity))
                aspects = raw_data['Aspects']
                #### data preprocess end 

                #### 속성 기반 분류
                data = {'input': {}, 'output': {}}
                ## preprocess data from line
                data['input']['text'] = text
                data['input']['domain'] = domain
                data['output']['general_polarity'] = generl_polarity
                data['output']['aspects'] = aspects
                ## preprocess data from line end
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 속성 기반 분류 end


                #### 도메인 식별
                data = {'input': None, 'output': None}
                ## preprocess data from line
                data['input'] = text
                data['output'] = domain
                ## preprocess data from line end
                task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 도메인 식별 end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 25/25 [00:00<00:00, 49.98it/s]
